In [1]:
#!pip3 install datasets

In [2]:
#!pip3 install transformers -U;

In [3]:
#!pip3 install accelerate -U

In [4]:
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn.metrics import classification_report
from sklearn.model_selection import cross_validate
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

import matplotlib.pyplot as plt

import torch
import tensorflow as tf

from sklearn.model_selection import train_test_split

import datasets
from datasets import Dataset, DatasetDict

from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer

c:\Users\jason\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# !git clone https://github.com/cse151a-DrugReviewAnalysis/DrugReviewAnalysis.git

In [6]:
# Get dataset for working locally
df_train = pd.read_csv("drugsTrain_processed_with_usefulCount.csv", sep="\t")
df_test = pd.read_csv("drugsTest_processed_with_usefulCount.csv", sep="\t")

In [7]:
# Get dataset when working on colab
# df_train = pd.read_csv("DrugReviewAnalysis/Model 3/drugsTrain_processed_with_usefulCount.csv", sep="\t")
# df_test = pd.read_csv("DrugReviewAnalysis/Model 3/drugsTest_processed_with_usefulCount.csv", sep="\t")

In [8]:
df_train = df_train.sample(frac=0.7, random_state=42).reset_index()
df_test = df_test.sample(frac=0.7, random_state=42).reset_index()

In [9]:
df_train.shape, df_test.shape

((111649, 8), (37240, 8))

In [10]:
df_val, df_test_new = train_test_split(df_test, test_size=0.6, random_state=42)

In [11]:
ds_train = Dataset.from_pandas(df_train[["processed_review", "rating"]])
ds_val = Dataset.from_pandas(df_val.reset_index()[["processed_review", "rating"]])
ds_test = Dataset.from_pandas(df_test_new.reset_index()[["processed_review", "rating"]])

In [12]:
ds = DatasetDict()

ds["train"] = ds_train
ds["validation"] = ds_val
ds["test"] = ds_test

In [13]:
ds

DatasetDict({
    train: Dataset({
        features: ['processed_review', 'rating'],
        num_rows: 111649
    })
    validation: Dataset({
        features: ['processed_review', 'rating'],
        num_rows: 14896
    })
    test: Dataset({
        features: ['processed_review', 'rating'],
        num_rows: 22344
    })
})

In [14]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [15]:
def preprocess(samples):
  text = samples["processed_review"]
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=128, return_attention_mask=True)
  encoding["labels"] = samples["rating"]
  return encoding

In [16]:
encoded_dataset = ds.map(preprocess, batched=True, remove_columns=ds['train'].column_names)

Map: 100%|██████████| 22344/22344 [00:02<00:00, 10070.04 examples/s]


In [17]:
encoded_dataset.set_format("torch")

In [18]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=1)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:

args = TrainingArguments(
    output_dir="/bert-regression", # for local
    # output_dir = "DrugReviewAnalysis/Model 3/bert-regression", # for colab
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="loss",
)

In [20]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  mse = mean_squared_error(labels, predictions)
  pred_modified = np.where(predictions < 10.0, np.round(predictions), 10.0)
  print(labels)
  acc = accuracy_score(labels, pred_modified)
  f1 = f1_score(labels, pred_modified, average="macro")
  return {"mse": mse, "accuracy-approximation": acc, "f1-approximation": f1}

In [21]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

c:\Users\jason\AppData\Local\Programs\Python\Python311\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [22]:
trainer.train()

  6%|▌         | 500/8725 [03:07<49:14,  2.78it/s] 

{'loss': 7.7243, 'grad_norm': 61.87055587768555, 'learning_rate': 1.8853868194842408e-05, 'epoch': 0.29}


 11%|█▏        | 1000/8725 [06:10<46:29,  2.77it/s] 

{'loss': 3.7083, 'grad_norm': 29.899028778076172, 'learning_rate': 1.7707736389684814e-05, 'epoch': 0.57}


 17%|█▋        | 1500/8725 [09:17<44:34,  2.70it/s]  

...

                                                   
100%|██████████| 8725/8725 [56:22<00:00,  3.28it/s]

{'eval_loss': 2.3601605892181396, 'eval_mse': 2.3601605892181396, 'eval_accuracy-approximation': 0.44716702470461867, 'eval_f1-approximation': 0.32351309376714515, 'eval_runtime': 29.602, 'eval_samples_per_second': 503.209, 'eval_steps_per_second': 7.871, 'epoch': 5.0}


100%|██████████| 8725/8725 [56:25<00:00,  2.58it/s]

{'train_runtime': 3385.0994, 'train_samples_per_second': 164.912, 'train_steps_per_second': 2.577, 'train_loss': 2.271469226389013, 'epoch': 5.0}


TrainOutput(global_step=8725, training_loss=2.271469226389013, metrics={'train_runtime': 3385.0994, 'train_samples_per_second': 164.912, 'train_steps_per_second': 2.577, 'train_loss': 2.271469226389013, 'epoch': 5.0})

In [23]:
trainer.evaluate()

100%|██████████| 233/233 [00:29<00:00,  7.98it/s]

[ 1.  1.  8. ...  1. 10.  4.]


{'eval_loss': 2.3601605892181396,
 'eval_mse': 2.3601605892181396,
 'eval_accuracy-approximation': 0.44716702470461867,
 'eval_f1-approximation': 0.32351309376714515,
 'eval_runtime': 29.5208,
 'eval_samples_per_second': 504.593,
 'eval_steps_per_second': 7.893,
 'epoch': 5.0}

In [24]:
# local
trainer.save_model("./bert-regression") 

# colab
# trainer.save_model("DrugReviewAnalysis/Model 3/bert-regression")

In [25]:
# local
# model = AutoModelForSequenceClassification.from_pretrained("bert-regression")

# colab
# model = AutoModelForSequenceClassification.from_pretrained("DrugReviewAnalysis/Model 3/bert-regression")

# trainer = Trainer(
#     model=model,
#     args=args,
#     train_dataset=encoded_dataset["train"],
#     eval_dataset=encoded_dataset["validation"],
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics
# )

In [26]:
trainer.evaluate(eval_dataset=encoded_dataset["test"])

  0%|          | 0/350 [00:00<?, ?it/s]

100%|██████████| 350/350 [00:43<00:00,  7.98it/s]

[10.  1.  1. ...  7.  6.  7.]


{'eval_loss': 2.3079092502593994,
 'eval_mse': 2.3079092502593994,
 'eval_accuracy-approximation': 0.4456229860365199,
 'eval_f1-approximation': 0.32198710079836285,
 'eval_runtime': 43.9864,
 'eval_samples_per_second': 507.975,
 'eval_steps_per_second': 7.957,
 'epoch': 5.0}